## Import Library

In [ ]:
from humpback import *

import pandas as pd
import numpy as np
from datetime import datetime
import ta

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

from backtesting import Backtest, Strategy

## Defining Functions

Functions in this section would be moved to humpback.py after testing for tidiness

## Defining Model

In [ ]:
class MyModel:
    def __init__(self):
        self.optimizer = 'adam'
        self.loss      = 'mean_squared_error'

    def trainModel(self, data_X, data_y, epochs: int = 20, batch_size: int = 32) -> None:
        self.trainScaler_X(data_X)
        self.trainScaler_y(data_y)

        self.model = Sequential()
        self.model.add(LSTM(units=50,
                            return_sequences=True,
                            input_shape=(data_X.shape[1],1)))
        self.model.add(Dropout(0.2))

        self.model.add(LSTM(units=50,
                            return_sequences=True))
        self.model.add(Dropout(0.2))

        self.model.add(LSTM(units=50,
                            return_sequences=True))
        self.model.add(Dropout(0.2))

        self.model.add(LSTM(units=50))
        self.model.add(Dropout(0.2))

        self.model.add(Dense(units=1))

        self.model.compile(optimizer=self.optimizer,
                           loss=self.loss)

        self.model.fit(self.scaler_X.transform(data_X),
                       self.scaler_y.transform(data_y),
                       epochs=epochs,
                       batch_size=batch_size)
    
    def trainScaler_X(self, data_X) -> None:
        self.scaler_X = MinMaxScaler()
        self.scaler_X.fit(data_X)

    def trainScaler_y(self, data_y) -> None:
        self.scaler_y = MinMaxScaler()
        self.scaler_y.fit(data_y)

    def predictModel(self, data_X):
        """output a predicted y given X

        Args:
            data_X (DataFrame): a pd.DataFrame of single row. Unscaled.

        Returns:
            unscaled prediction
        """
        scaled_X = self.scaler_X.transform(data_X)
        predict = self.model.predict(scaled_X)
        return self.scaler_y.inverse_transform(predict)
    
    def get_model(self):
        return self.model

## Defining Strategy

In [ ]:
class MyStr(Strategy):
    # Variables:
    train_size = 0.7
    buy_threshold  = .02
    sell_threshold = .02


    def init(self):
        # Declare indicators you will use in the strategy:
        self.getData()
        self.model_init()

    def next(self):
        if len(self.data) < len(self.y_train):
            return

        X = featureGeneration(self.data.df.iloc[-120:])
        forecast = self.model.predictModel(X.iloc[[-1]])[0]
        
        if forecast > self.buy_threshold and not self.position.is_long:
            tp = self.data.Close[-1] + 2 * X['ATR'][-1]
            sl = self.data.Close[-1] - 2 * X['ATR'][-1]
            self.buy(size=.2, tp=tp, sl=sl)

        if forecast < self.sell_threshold and not self.position.is_short:
            tp = self.data.Close[-1] - 2 * X['ATR'][-1]
            sl = self.data.Close[-1] + 2 * X['ATR'][-1]
            self.sell(size=.2, tp=tp, sl=sl)

########################################

    def getData(self):
        data_train, data_test = train_test_split(self.data.df,
                                                 train_size=self.train_size,
                                                 shuffle=False)
        data_train, data_test = [getReturn(data, 'Close') for data in [data_train, data_test]]
        data_train, data_test = [featureGeneration(data)  for data in [data_train, data_test]]
        self.X_train, self.y_train = getXy(data_train, 'Return')
        self.X_test , self.y_test  = getXy(data_test , 'Return')

    def model_init(self):
        self.model = MyModel()
        self.model.trainModel(self.X_train, self.y_train, epochs=5)

    def get_model(self):
        return self.model.get_model()


## Parameters

This section contains all the parameters.

In [ ]:
# This chunk specifies the data

symbol = 'BTCUSDT'
interval = '1h'
start_str = int(datetime(2020,1,1,0,0).timestamp() * 1000)
end_str    = int(datetime(2020,12,31,0,0).timestamp() * 1000)
# end_str    = int(datetime(2023,12,31,0,0).timestamp() * 1000)

## Obtain data

This section outputs training and testing data for the use of the following sections.

In [ ]:
client = connectBinanceAPI()

In [ ]:
data_raw = getBinanceData(client=client,
                          symbol=symbol,
                          interval=interval,
                          start_str=start_str,
                          end_str=end_str)

## Backtesting

In [ ]:
bt    = Backtest(data_raw, MyStr, cash = 1000000, commission=0.00075, margin=.05)
stats = bt.run()

## Results

In [ ]:
bt.plot()

In [ ]:
stats.tail()

In [ ]:
stats['_equity_curve']